In [1]:
import googlemaps
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from textwrap import shorten
import requests

from bs4 import BeautifulSoup

In [2]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

In [3]:
df = pd.read_csv('New projects in Gowanus this year - Map_Data.csv')

In [4]:
df['geo_address'] = df['Address'] + ' Gowanus, New York City'

In [5]:
def geocode(add):
    g = gmaps_key.geocode(add)
    lat = g[0]["geometry"]["location"]["lat"]
    lng = g[0]["geometry"]["location"]["lng"]
    return (lat, lng)

df['geocoded'] = df['geo_address'].apply(geocode)

In [6]:
df['geocoded'] = df['geocoded'].astype(str)
df[['lat', 'lon']] = df['geocoded'].str.strip('()').str.split(', ', expand=True)
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)

## Image URL Collector

In [7]:
df['image_links'] = ''
links = []
for url in df['TRD Link']:
    try:
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
        images = soup.find('img',{'class':'MainArtImage_image__2YtGm'})
        images = images['src']
        links.append(images)
    except:
        links.append(None)

df['image_links'] = links

In [8]:
print(df['TRD Link'].iloc[2])

nan


In [9]:
print(df['image_links'].iloc[2])

None


In [10]:
# df.at[20,'image_img']=('<img src="Aimco.jpg" width="256" height="156">')
df['image_links'].iloc[1]

'https://static.therealdeal.com/wp-content/uploads/2021/11/main-Vorea-plans-2-buildings-628-units-in-Gowanus.jpg'

In [11]:
# df.at[4,'image_links']=('https://therealdeal.com/wp-content/uploads/2019/10/trd-logo-16x9.jpg')

In [12]:
df["Owner/Developer"]

0                           Quinlan Development Group
1                       Vorea Group, Domain Companies
2                                  Crown Acquisitions
3                       Vorea Group, Domain Companies
4                             Tavros Capital Partners
5                             Tavros Capital Partners
6                                  PMG, Carlyle Group
7                                  PMG, Carlyle Group
8          Tavros Capital Partners, Charney Companies
9     Tavros Capital Partners, Avery Hall Investments
10                             Avery Hall Investments
11                               Heron Fourth Ave LLC
12                                  Orange Management
13                             Avery Hall Investments
14                              Monadnock Development
15                                       RFR Holdings
16                  Midwood Investments & Development
17                                             Domain
18                          

In [13]:
df["short_description"] = df["Owner/Developer"].apply(lambda s: shorten(s, width=20, placeholder=""))
df['remaining_desc'] = df.apply(lambda row : row['Owner/Developer'].replace(str(row['short_description']), ''), axis=1)
df['remaining_desc']

df["description_link"] = '<a href="' + df["TRD Link"] + '" target="_blank" rel="noopener noreferrer">' + df["short_description"] + "</a>" + df["remaining_desc"]
df['description_link']

0     <a href="https://therealdeal.com/2021/10/20/qu...
1     <a href="https://therealdeal.com/2021/11/08/fi...
2                                                   NaN
3     <a href="https://therealdeal.com/2021/11/08/fi...
4                                                   NaN
5                                                   NaN
6     <a href="https://therealdeal.com/new-york/2023...
7                                                   NaN
8     <a href="https://therealdeal.com/new-york/2022...
9     <a href="https://therealdeal.com/new-york/2021...
10                                                  NaN
11                                                  NaN
12    <a href="https://therealdeal.com/new-york/2023...
13    <a href="https://therealdeal.com/new-york/2019...
14    <a href="https://therealdeal.com/new-york/2021...
15    <a href="https://therealdeal.com/new-york/2022...
16                                                  NaN
17    <a href="https://therealdeal.com/new-york/

In [14]:
df['description_link']

0     <a href="https://therealdeal.com/2021/10/20/qu...
1     <a href="https://therealdeal.com/2021/11/08/fi...
2                                                   NaN
3     <a href="https://therealdeal.com/2021/11/08/fi...
4                                                   NaN
5                                                   NaN
6     <a href="https://therealdeal.com/new-york/2023...
7                                                   NaN
8     <a href="https://therealdeal.com/new-york/2022...
9     <a href="https://therealdeal.com/new-york/2021...
10                                                  NaN
11                                                  NaN
12    <a href="https://therealdeal.com/new-york/2023...
13    <a href="https://therealdeal.com/new-york/2019...
14    <a href="https://therealdeal.com/new-york/2021...
15    <a href="https://therealdeal.com/new-york/2022...
16                                                  NaN
17    <a href="https://therealdeal.com/new-york/

In [15]:
def popup_html(row):
    i = row
    address = df['Address'].iloc[i]
    Owner_or_Developer = df['Owner/Developer'].iloc[i] if pd.isnull(df['description_link'].iloc[i]) else df['description_link'].iloc[i]
    total_const_floor_area = df['Total Construction Floor Area'].iloc[i]
    image = df['image_links'].iloc[i]
    
    image_html = '<img src="{}" width="256" height="156"><br>______________________________________<br>'.format(image) if pd.notnull(image) else ''
    
    html = '''<!DOCTYPE html>
    <html>''' + \
    image_html + \
    '''<strong>Address: </strong>{}'''.format(address) + '''<br>
    <strong>Developer: </strong>{}'''.format(Owner_or_Developer) + '''<br>
    <strong>Total Construction Floor Area: </strong>{}'''.format(total_const_floor_area) + '''
    </html>
    '''
    return html

In [16]:
import branca

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(),zoom_start=15)

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'New building applications, preserved affordable housing provided by the Gowanus rezoning')

for i in range(0,len(df)):
    html = popup_html(i)
    iframe = branca.element.IFrame(html=html, width=256)
    popup = folium.Popup(folium.Html(html, script=True), max_width=256)
    folium.Marker([df['lat'].iloc[i],df['lon'].iloc[i]],
                 popup=popup).add_to(m)

m.get_root().html.add_child(folium.Element(title_html))
m.fit_bounds(bounds=df[['lat','lon']].mean().to_list(),max_zoom=15)
# m.add_to(f)
m

In [17]:
m.save('index.html')

## Map URL Snagger

In [18]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/New_Gowanus_Projects_July_2023
